In [1]:
from sqlalchemy import create_engine
import pandas as pd
from getpass import getpass  # To get the password without showing the input

import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
#sns.set_theme(style="whitegrid")

from sklearn.metrics import confusion_matrix
from sklearn.metrics import plot_confusion_matrix
from sklearn.metrics import cohen_kappa_score

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression     # despite its name it is used for classification
from sklearn.metrics import accuracy_score

#### **Task**

*Create a Python connection with SQL database and retrieve the results of the last two queries (also mentioned below) as dataframes:*

- *Check the number of rentals for each customer for May*
- *Check the number of rentals for each customer for June*

In [2]:
password = getpass("INSERT PASSWORD: ")

In [3]:
# creating a connection string: defining the connection string as a relation among dialect, driver, username, password host:port and database

connection_string = 'mysql+pymysql://root:' + password + '@localhost/sakila'
engine = create_engine(connection_string)       # then initialising an engine based on the connection string I've created

In [4]:
# I created a join between the tables rental and customer on mysql in saqila, in order to get prettier tables, containing the customers' full names

query = 'SELECT C.customer_id as "customer_id", C.first_name as "first_name", C.last_name as "last_name", count(RM.rental_id) as "may_num_rentals" \
FROM rentals_may RM \
INNER JOIN customer C \
ON RM.customer_id = C.customer_id \
GROUP BY customer_id \
ORDER BY may_num_rentals'
rentals_may = pd.read_sql_query(query, engine)      # essentially the command to extract the info from the database on sql
rentals_may.head()


,customer_id,first_name,last_name,may_num_rentals
0,459,TOMMY,COLLAZO,1
1,333,ANDREW,PURDY,1
2,456,RONNIE,RICKETTS,1
3,88,BONNIE,HUGHES,1
4,532,NEIL,RENNER,1


In [5]:
query = 'SELECT C.customer_id as "customer_id", C.first_name as "first_name", C.last_name as "last_name", count(RJ.rental_id) as "june_num_rentals" \
FROM rentals_june RJ \
INNER JOIN customer C \
ON RJ.customer_id = C.customer_id \
GROUP BY customer_id \
ORDER BY june_num_rentals'
rentals_june = pd.read_sql_query(query, engine)
rentals_june.head()

,customer_id,first_name,last_name,june_num_rentals
0,370,WAYNE,TRUONG,1
1,168,REGINA,BERRY,1
2,369,FRED,WHEAT,1
3,302,MICHAEL,SILVERMAN,1
4,394,CHRIS,BROTHERS,1


#### **Task**

- *Write a function that checks if customer borrowed more or less films in the month of June as compared to May.*

In [24]:
# comp_df = pd.concat([rentals_may,rentals_june]) # could include all the available values, but inappropriate, because customers who have been active only one of the months aren't relevant

In [10]:
# I want to avoid creating a table with duplicate columns, so I will create a data frame containing only those columns from rentals_may that are unique and then merge the new df with rentals_june

no_dups_may = rentals_may.columns.difference(rentals_june.columns)
no_dups_may_df = rentals_may[no_dups_may]
no_dups_may_df

,may_num_rentals
0,1
1,1
2,1
3,1
4,1
...,...
515,6
516,6
517,7
518,7


In [11]:
# merging my 2 dataframes

comp_df = no_dups_df.merge(rentals_june, left_index=True, right_index=True, how='inner')       # comp_df will only contain rows that exist in both original dfs, because otherwise no comparison can be made
comp_df

,may_num_rentals,customer_id,first_name,last_name,june_num_rentals
0,1,370,WAYNE,TRUONG,1
1,1,168,REGINA,BERRY,1
2,1,369,FRED,WHEAT,1
3,1,302,MICHAEL,SILVERMAN,1
4,1,394,CHRIS,BROTHERS,1
...,...,...,...,...,...
515,6,130,CHARLOTTE,HUNTER,6
516,6,319,RONALD,WEINER,6
517,7,85,ANNE,POWELL,6
518,7,514,FRANKLIN,TROUTMAN,6


In [12]:
# creating a column with values 0 and 1, where 0 will mean that the customer has rented more times in May and 1 that they rented more times in June

comp_df['more_in_june'] = 0
comp_df['more_in_june'][comp_df['may_num_rentals']<comp_df['june_num_rentals']] = 1

C:\Users\vagli\AppData\Local\Temp\ipykernel_1984\2667484314.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  comp_df['more_in_june'][comp_df['may_num_rentals']<comp_df['june_num_rentals']] = 1


#### I don't like the arrangement of columns in my dataframe, so I will change it to something that is easier to read

In [13]:
comp_df.columns

Index(['may_num_rentals', 'customer_id', 'first_name', 'last_name',
       'june_num_rentals', 'more_in_june'],
      dtype='object')

In [14]:
comp_df = comp_df[['customer_id', 'first_name', 'last_name', 'may_num_rentals', 'june_num_rentals', 'more_in_june']]
comp_df.sample(20)

,customer_id,first_name,last_name,may_num_rentals,june_num_rentals,more_in_june
90,596,ENRIQUE,FORSYTHE,1,2,1
297,270,LEAH,CURTIS,2,4,1
99,407,DALE,RATCLIFF,1,2,1
118,551,CLAYTON,BARBEE,1,2,1
447,24,KIMBERLY,LEE,3,5,1
258,19,RUTH,MARTINEZ,2,3,1
408,221,BESSIE,MORRISON,3,5,1
281,287,BECKY,MILES,2,4,1
254,15,HELEN,HARRIS,2,3,1
93,73,BEVERLY,BROOKS,1,2,1


#### **The last column contains the information about whether each respective customer has rented more films in May or June**

In [15]:
comp_df.isna().sum()

customer_id         0
first_name          0
last_name           0
may_num_rentals     0
june_num_rentals    0
more_in_june        0
dtype: int64

In [16]:
comp_df['customer_id'].duplicated().sum()

0

#### The tables merged well. There were no duplicate rows and no NaNs

In [18]:
# as a bonus I created a simple function that allows me to check out the activity of specific customers, based on their customer_id, by simply inputting it when asked by the running function

def how_active():
    x = int(input("Please insert customer_id number: "))
    customer_activity = comp_df.loc[comp_df['customer_id'] == x]
    return customer_activity


In [19]:
cust_573 = how_active()        # selecting random customer_id from the frame to test the function
cust_573

,customer_id,first_name,last_name,may_num_rentals,june_num_rentals,more_in_june
332,573,BYRON,BOX,2,4,1


##### *For example Byron Box borrowed more films in June in comparison to May*